In [1]:
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split


## Imports

In [247]:
### Pickled data
import pickle
cleaned_data = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\cleaned_data_Maor.pkl")
X_train = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\X_train.pkl")
X_test = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\X_test.pkl")
y_train = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\y_train.pkl")
y_test = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\y_test.pkl")
predictions = pd.read_csv(r'C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\predictions\predictions.csv')

In [3]:
raw_data = pd.read_csv(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\raw\train_dataset_full.csv")

In [249]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape, predictions.shape, cleaned_data.shape, raw_data.shape

((293307, 15),
 (73327, 15),
 (293307,),
 (73327,),
 (73160, 1),
 (365798, 14),
 (389163, 15))

In [253]:
X_test.isna().sum()

product                   0
campaign_id               0
webpage_id                0
gender                    0
age_level                 0
user_depth                0
city_development_index    0
var_1                     0
product_category          0
Month                     0
Day                       0
Hour                      0
Minute                    0
weekday                   0
campaign_duration_days    0
dtype: int64

In [225]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape, predictions.shape, cleaned_data.shape, raw_data.shape

((292638, 13),
 (73160, 13),
 (292638,),
 (73160,),
 (73160, 1),
 (365798, 14),
 (389163, 15))

In [506]:
#find how many duplicated rows we have
duplicates = raw_data[raw_data.duplicated()]
duplicates.shape

(19510, 15)

In [250]:
y_train.isna().sum()    

0

## Feature Engineering

In [ ]:
import pandas as pd
import numpy as np

def feature_generation(df):
    """Generate date/time features and fill missing values in a faster, 
       more scalable way without repeated group-based ffill/bfill.
    """
    df = df.copy()
    
    # If not already a datetime, convert:
    # df['DateTime'] = pd.to_datetime(df['DateTime'])
    
    # -- 1) Create date/time features --
    df['Month'] = df['DateTime'].dt.month
    df['Day'] = df['DateTime'].dt.day
    df['Hour'] = df['DateTime'].dt.hour
    df['Minute'] = df['DateTime'].dt.minute
    df['weekday'] = df['DateTime'].dt.weekday
    
    # -- 2) Drop unnecessary columns --
    df.drop(columns=['DateTime', 'session_id'], inplace=True, errors='ignore')
    
    # -- 3) Make user_id a consistent type --
    # (Strings are often safer keys for merges.)
    df['user_id'] = df['user_id'].astype(str)
    
    # -- 4) Identify columns to fill by median vs. mode --
    #    (You can tune these lists as needed.)
    columns_to_fill_median = ['Month', 'Day', 'Hour', 'Minute', 'weekday']
    columns_to_fill_mode = [
        'product', 'campaign_id', 'webpage_id', 'gender', 
        'age_level', 'user_depth', 'city_development_index', 
        'var_1', 'product_category'
    ]
    
    # Keep only columns that actually exist in df
    columns_to_fill_median = [c for c in columns_to_fill_median if c in df.columns]
    columns_to_fill_mode = [c for c in columns_to_fill_mode if c in df.columns]
    
    # -- 5) Precompute the user-level medians/modes in one pass each --
    if columns_to_fill_median:
        median_df = (
            df.groupby('user_id')[columns_to_fill_median]
            .median()
            .reset_index()
        )
    
    # Mode can be tricky (pandas mode can return multiple values).
    # We'll define a custom aggregator that picks the first mode if multiple modes exist.
    def agg_mode(s):
        m = s.mode(dropna=True)
        return m.iloc[0] if len(m) > 0 else np.nan
        
    if columns_to_fill_mode:
        mode_df = (
            df.groupby('user_id')[columns_to_fill_mode]
            .agg(agg_mode)
            .reset_index()
        )
        
    # -- 6) Merge those statistics back to df --
    # This is usually much more performant than repeated group transforms:
    if columns_to_fill_median:
        df = df.merge(
            median_df, 
            on='user_id', 
            suffixes=('', '_median')
        )
    if columns_to_fill_mode:
        df = df.merge(
            mode_df, 
            on='user_id', 
            suffixes=('', '_mode')
        )
        
    # -- 7) Fill missing values in df using the merged median/mode --
    if columns_to_fill_median:
        for col in columns_to_fill_median:
            df[col] = df[col].fillna(df[f'{col}_median'])
            df.drop(columns=[f'{col}_median'], inplace=True, errors='ignore')
            
    if columns_to_fill_mode:
        for col in columns_to_fill_mode:
            df[col] = df[col].fillna(df[f'{col}_mode'])
            df.drop(columns=[f'{col}_mode'], inplace=True, errors='ignore')
    
    for col in columns_to_fill_median:
        df[col] = df[col].fillna(df[col].median())

    for col in columns_to_fill_mode:
        df[col] = df[col].fillna(df[col].mode()[0])
    
    df.drop(columns=['user_id','user_group_id'], inplace=True, errors='ignore')
            
    return df

# Example usage:
X_train_u = feature_generation(X_train)
X_test_u  = feature_generation(X_test)


Month
Day
Hour
Minute
weekday
product
campaign_id
webpage_id
gender
age_level
user_depth
city_development_index
var_1
product_category
Month
Day
Hour
Minute
weekday
product
campaign_id
webpage_id
gender
age_level
user_depth
city_development_index
var_1
product_category


In [14]:
def fill_missing_with_mode(df: pd.DataFrame, columns: list):
    df = df.copy()
    for column in columns:
        if column in df.columns:
            mode_value = df[column].mode()[0]  # Calculate the mode
            df[column] = df[column].fillna(mode_value)  # Fill missing values with the mode
    return df

def fill_missing_with_median(df: pd.DataFrame, columns: list):
    df = df.copy()
    for column in columns:
        if column in df.columns:
            median_value = df[column].median()  # Calculate the median
            df[column] = df[column].fillna(median_value)  # Fill missing values with the median
    return df   

In [15]:
X_train.columns

Index(['session_id', 'DateTime', 'user_id', 'product', 'campaign_id',
       'webpage_id', 'user_group_id', 'gender', 'age_level', 'user_depth',
       'city_development_index', 'var_1', 'product_category'],
      dtype='object')

In [168]:
def fill_missing_with_mode(df: pd.DataFrame, columns: list):
    df = df.copy()
    for column in columns:
        if column in df.columns:
            mode_value = df[column].mode()[0]  # Calculate the mode
            df[column] = df[column].fillna(mode_value)  # Fill missing values with the mode
    return df

def fill_missing_with_median(df: pd.DataFrame, columns: list):
    df = df.copy()
    for column in columns:
        if column in df.columns:
            median_value = df[column].median()  # Calculate the median
            df[column] = df[column].fillna(median_value)  # Fill missing values with the median
    return df   

def feature_generation(df, use_missing_with_mode=False, get_dumm=False,catb = True):
    df = df.copy()
    df['Month'] = df['DateTime'].dt.month
    df['Day'] = df['DateTime'].dt.day
    df['Hour'] = df['DateTime'].dt.hour
    df['Minute'] = df['DateTime'].dt.minute
    df['weekday'] = df['DateTime'].dt.weekday
    
    columns_to_fill_median = ["Month", "Day", "Hour", "Minute", "weekday"]
    if use_missing_with_mode:
        columns_to_fill_mode = ["product", "campaign_id", "webpage_id", "user_group_id", "gender", "age_level", "user_depth", "city_development_index", "var_1", "product_category",
                     "month", "day", "hour"]
        df = fill_missing_with_mode(df, columns_to_fill_mode)
    df['campaign_id'] = df['campaign_id'].fillna(df['campaign_id'].mode()[0])
    df = fill_missing_with_median(df, columns_to_fill_median)
    print(df.isna().sum())
    df['start_date'] = df.groupby('campaign_id',observed=True)['DateTime'].transform('min')
    df['campaign_duration'] = df['DateTime'] - df['start_date']
    df['campaign_duration_days'] = df['campaign_duration'].dt.total_seconds() / (3600) #hour did not improve, day didn't improve either
    df['campaign_duration_days'] = df['campaign_duration_days'].fillna(
    df.groupby('campaign_id',observed=True)['campaign_duration_days'].transform(lambda x: x.mode().iloc[0])).astype(int)
    df.drop(columns=['DateTime', 'start_date', 'campaign_duration','session_id','user_id','user_group_id'], inplace=True)
    if get_dumm:
        columns_to_d = ["product", "campaign_id", "webpage_id", "product_category", "gender"]
        df = pd.get_dummies(df, columns=columns_to_d)


    return df
    
    
    

In [218]:
def fill_missing_with_mode(df: pd.DataFrame, columns: list):
    df = df.copy()
    for column in columns:
        if column in df.columns:
            mode_value = df[column].mode()[0]  # Calculate the mode
            df[column] = df[column].fillna(mode_value)  # Fill missing values with the mode
    return df

def fill_missing_with_median(df: pd.DataFrame, columns: list):
    df = df.copy()
    for column in columns:
        if column in df.columns:
            median_value = df[column].median()  # Calculate the median
            df[column] = df[column].fillna(median_value)  # Fill missing values with the median
    return df

def determine_categorical_features(df: pd.DataFrame, cat_features: list = None):

    if cat_features:
        cat_features = [col for col in cat_features if col in df.columns]
    else:
        cat_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

    for col in cat_features:
        if col in df.columns:
            # Ensure column is treated as category
            df[col] = df[col].astype("category")

            # Add "missing" only if it's not already a category
            if "missing" not in df[col].cat.categories:
                df[col] = df[col].cat.add_categories("missing")

            # Fill missing values with "missing"
            df[col] = df[col].fillna("missing")

    return cat_features

def feature_generation2(df, use_missing_with_mode=False, get_dumm=False, catb=True, cat_features=None):

    df = df.copy()

    # Generate time-based features
    df['Month'] = df['DateTime'].dt.month
    df['Day'] = df['DateTime'].dt.day
    df['Hour'] = df['DateTime'].dt.hour
    df['Minute'] = df['DateTime'].dt.minute
    df['weekday'] = df['DateTime'].dt.weekday

    # Handle categorical features if `catb` is True
    if catb:
        cat_features = determine_categorical_features(df, cat_features)

    # Fill missing values
    if use_missing_with_mode:
        print("Filling missing values with mode")
        columns_to_fill_mode = ["product", "campaign_id", "webpage_id", "user_group_id", "gender", "age_level", "user_depth", "city_development_index", "var_1", "product_category",
                                "Month", "Day", "Hour"]
        df = fill_missing_with_mode(df, columns_to_fill_mode)

    columns_to_fill_median = ["Month", "Day", "Hour", "Minute", "weekday", "city_development_index", "age_level", "user_depth"]
    df['campaign_id'] = df['campaign_id'].fillna(df['campaign_id'].mode()[0]) #userid and sessionid have nas. What can we do else?
    df['var_1'] = df['var_1'].fillna(df['var_1'].mode()[0])
    df = fill_missing_with_median(df, columns_to_fill_median)

    # Generate campaign-based features
    df['start_date'] = df.groupby('campaign_id', observed=True)['DateTime'].transform('min')
    df['campaign_duration'] = df['DateTime'] - df['start_date']
    df['campaign_duration_days'] = df['campaign_duration'].dt.total_seconds() / (3600*24)
    df['campaign_duration_days'] = df['campaign_duration_days'].fillna(
        df.groupby('campaign_id', observed=True)['campaign_duration_days'].transform(lambda x: x.mode().iloc[0])).astype(int)

    # Drop unnecessary columns
    df.drop(columns=['DateTime', 'start_date', 'campaign_duration', 'session_id', 'user_id', 'user_group_id'], inplace=True)

    # One-hot encoding if `get_dumm` is True
    if get_dumm:
        columns_to_d = ["product", "campaign_id", "webpage_id", "product_category", "gender"]
        df = pd.get_dummies(df, columns=columns_to_d)

    return df


In [219]:
X_train2 = feature_generation2(X_train)
X_test2 = feature_generation2(X_test)
cat_features = determine_categorical_features(X_train2)

In [242]:
columns_to_d = ["product", "campaign_id", "webpage_id", "product_category", "gender"]
X_train_d = pd.get_dummies(X_train, columns=columns_to_d)
X_test_d = pd.get_dummies(X_test, columns=columns_to_d)

In [212]:
X_train2['gender'].value_counts()

gender
Male       248397
Female      32776
missing     11465
Name: count, dtype: int64

In [96]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(n_estimators=100, random_state=42,class_weight='balanced')
rf.fit(X_train2, y_train)
y_pred = rf.predict(X_test2)

f1_score(y_test, y_pred)

0.03980257920713262

In [93]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Apply SMOTE to balance the training set
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train2, y_train)

# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_smote, y_train_smote)

# Predict and evaluate
y_pred2 = rf.predict(X_test2)



In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
#adjuct the balance of the classes
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42, solver='liblinear',
                           penalty='l2', C=0.1)
model.fit(X_train_filled_d, y_train)

y_pred_LR = model.predict(X_test_filled_d)
print(f1_score(y_test, y_pred_LR))

0.1433358256486943


In [244]:
#import cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB

from sklearn.model_selection import cross_val_score
model = ComplementNB()
cross_val_score(model, X_train_d, y_train, cv=5, scoring='f1')

array([0.14267398, 0.14025811, 0.14581596, 0.13800516, 0.14146667])

In [245]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB
from sklearn.metrics import f1_score, classification_report
model = ComplementNB()
model.fit(X_train_d, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_d)
f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1}")
print(classification_report(y_test, y_pred))

F1-score: 0.14497621046739434
              precision    recall  f1-score   support

         0.0       0.94      0.59      0.73     68818
         1.0       0.08      0.51      0.14      5063

    accuracy                           0.59     73881
   macro avg       0.51      0.55      0.44     73881
weighted avg       0.88      0.59      0.69     73881



**Baseline? :()**

In [163]:
cleaned_data['is_click'].value_counts()[0]/cleaned_data['is_click'].value_counts().sum()

0.9323260378678943

In [164]:
a = 1- cleaned_data['is_click'].value_counts()[0]/cleaned_data['is_click'].value_counts().sum()

In [568]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 292638 entries, 196 to 123594
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   session_id              292076 non-null  float64       
 1   DateTime                292049 non-null  datetime64[ns]
 2   user_id                 292638 non-null  float64       
 3   product                 292584 non-null  category      
 4   campaign_id             292063 non-null  category      
 5   webpage_id              292048 non-null  category      
 6   user_group_id           280649 non-null  category      
 7   gender                  281173 non-null  category      
 8   age_level               281178 non-null  float64       
 9   user_depth              280646 non-null  float64       
 10  city_development_index  213253 non-null  float64       
 11  var_1                   292591 non-null  float64       
 12  product_category        292139 no

In [569]:
X_train1 = X_train.drop(columns=['session_id', 'DateTime', 'user_id'])

In [425]:
from catboost import CatBoostClassifier
def objective(trial):
    # --- 1) Suggest hyperparams ---
    # Example search space: Feel free to expand or tune ranges
    params = {
        "iterations": 1000,
        "depth": trial.suggest_int("depth", 4, 8),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 100),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        # Commonly used knobs; random_strength, subsample, etc. can also be tuned
        "eval_metric": "F1",
        "random_seed": 42,
        "auto_class_weights": "Balanced",
        "verbose": 0  # Keep CatBoost silent
    }

    # --- 2) Create CatBoost model ---
    model = CatBoostClassifier(**params)

    # --- 3) Train/Validation split ---
    X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(
        X_train1,
        y_train,
        test_size=0.2,
        random_state=42,
        stratify=y_train
    )

    # --- 4) Train ---
    model.fit(
        X_train_sub,
        y_train_sub,
        cat_features=cat_features,
        eval_set=(X_val_sub, y_val_sub),
        early_stopping_rounds=50,
        use_best_model=True
    )

    # --- 5) Predict ---
    y_pred_val = model.predict(X_val_sub)

    # --- 6) Evaluate ---
    f1 = f1_score(y_val_sub, y_pred_val)

    # Return the F1 (Optuna will try to maximize this)
    return f1


In [89]:
# Create study that aims to maximize F1
import optuna
study = optuna.create_study(direction="maximize")

# Optimize over 'objective' for a certain number of trials
study.optimize(objective, n_trials=20, show_progress_bar=True)


[I 2025-01-23 21:27:34,790] A new study created in memory with name: no-name-8ce8d20c-b456-437c-8c24-278df2096bb7


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-01-23 21:27:52,779] Trial 0 finished with value: 0.14241381636645753 and parameters: {'iterations': 700, 'depth': 7, 'learning_rate': 0.09744541619010162, 'l2_leaf_reg': 4.6845889641307, 'bagging_temperature': 0.6083577649428933}. Best is trial 0 with value: 0.14241381636645753.
[I 2025-01-23 21:29:00,012] Trial 1 finished with value: 0.14715379312846932 and parameters: {'iterations': 900, 'depth': 8, 'learning_rate': 0.06056087732149749, 'l2_leaf_reg': 44.82689039760325, 'bagging_temperature': 0.3545388999625765}. Best is trial 1 with value: 0.14715379312846932.
[I 2025-01-23 21:29:18,804] Trial 2 finished with value: 0.14246593421096043 and parameters: {'iterations': 400, 'depth': 6, 'learning_rate': 0.012942977089435637, 'l2_leaf_reg': 23.56083041913574, 'bagging_temperature': 0.4558729858095575}. Best is trial 1 with value: 0.14715379312846932.
[I 2025-01-23 21:29:34,801] Trial 3 finished with value: 0.14265668849391955 and parameters: {'iterations': 700, 'depth': 7, 'learn

In [237]:
# Import necessary libraries
import pandas as pd
import logging
from pathlib import Path
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold

# Define the ModelTrainer class
class ModelTrainer:
    def __init__(self, data_dir: False, model_name: str = "catboost", cat_features: list = None):
        #self.data_dir = Path(data_dir)
        self.model_name = model_name
        self.cat_features = cat_features

        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def load_data(self):
        self.logger.info(f"Loading preprocessed data from {self.data_dir}...")
        X_train = pd.read_pickle(self.data_dir / "X_train.pkl")
        y_train = pd.read_pickle(self.data_dir / "y_train.pkl").squeeze()
        return X_train, y_train

    def determine_categorical_features(self, X_train: pd.DataFrame):
        if self.cat_features:
            cat_features = [col for col in self.cat_features if col in X_train.columns]
        else:
            cat_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

        self.logger.info(f"Categorical features: {cat_features}")
        return cat_features


    def cross_validate_model(self, X_train: pd.DataFrame, y_train: pd.Series, cat_features: list, cv: int = 5):
        if self.model_name == 'catboost':
            model = CatBoostClassifier(
                random_seed=42, verbose=0, eval_metric='F1',
                cat_features=cat_features, class_weights=[1, 10]
            )
        else:
            raise ValueError(f"Unsupported model: {self.model_name}")

        self.logger.info(f"Performing {cv}-fold cross-validation...")
        skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

        fold_scores = []
        for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
            self.logger.info(f"Processing fold {fold + 1}...")

            X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

            self.logger.info(f"Validation set shape for fold {fold + 1}: {X_fold_val.shape}")

            model.fit(X_fold_train, y_fold_train, eval_set=(X_fold_val, y_fold_val), use_best_model=True)

            fold_score = model.best_score_['validation']['F1']
            fold_scores.append(fold_score)

            self.logger.info(f"Fold {fold + 1} F1 score: {fold_score}")

        mean_cv_score = sum(fold_scores) / len(fold_scores)
        self.logger.info(f"Mean cross-validation F1 score: {mean_cv_score}")
        return mean_cv_score

    def train_model(self, X_train: pd.DataFrame, y_train: pd.Series, cat_features: list, val_size: float = 0.2):
        if self.model_name == 'catboost':
            model = CatBoostClassifier(
            random_seed=42,
            verbose=100,
            eval_metric='F1',
            cat_features=cat_features,
           #auto_class_weights='Balanced',
            max_depth=5,
           #colsample_bylevel=0.7,
            class_weights=[1, 1/a],
            bagging_temperature=0.4,
            grow_policy='SymmetricTree',
          # one_hot_max_size = 40,
          # learning_rate=0.1,
         #  subsample=.67,   #lower subsample showed progress
         #  max_leaves= 64, #only with lossguide
            bootstrap_type = "Bayesian", #Bayesian uses the posterior probability of the object 
                                        #to sample the trees in the growing process. Good for regularization and overfitting control.
          # bootstrap_type='Bernoulli', #Bernoulli is Stochastic Gradient Boosting on random subsets of features, faster and less overfitting
            early_stopping_rounds=100,
            )

        else:
            raise ValueError(f"Unsupported model: {self.model_name}")

        self.logger.info(f"Training {self.model_name} model...")
        X_train.drop(columns=['session_id', 'DateTime', 'user_id'], inplace=True, errors='ignore')

        X_train_final, X_valid, y_train_final, y_valid = train_test_split(
            X_train, y_train, test_size=val_size, random_state=42)
        

        self.logger.info(f"Training {self.model_name} model with validation set...")
        model.fit(X_train_final, y_train_final, eval_set=(X_valid, y_valid), use_best_model=True)

        return model

# Interactive Workflow for Jupyter Notebook
# Define the data directory
DATA_DIR = "path/to/data"  # Replace with your actual path

# Initialize ModelTrainer
trainer = ModelTrainer(DATA_DIR,model_name="catboost")
cat_features = trainer.determine_categorical_features(X_train)



INFO:__main__:Categorical features: ['product', 'campaign_id', 'webpage_id', 'gender', 'product_category']


In [246]:
a

0.06767396213210575

In [238]:
model = trainer.train_model(X_train, y_train, cat_features)



INFO:__main__:Training catboost model...
INFO:__main__:Training catboost model with validation set...


Learning rate set to 0.122239
0:	learn: 0.5681503	test: 0.5585585	best: 0.5585585 (0)	total: 191ms	remaining: 3m 10s
100:	learn: 0.6083793	test: 0.5910177	best: 0.5937677 (39)	total: 26.1s	remaining: 3m 52s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.5937676693
bestIteration = 39

Shrink model to first 40 iterations.


In [239]:
y_pred = model.predict(X_test)

In [240]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
print(classification_report(y_test, y_pred,digits=4))

              precision    recall  f1-score   support

         0.0     0.9462    0.4857    0.6419     68818
         1.0     0.0820    0.6245    0.1450      5063

    accuracy                         0.4952     73881
   macro avg     0.5141    0.5551    0.3934     73881
weighted avg     0.8870    0.4952    0.6078     73881



In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# After training your CatBoost model:
y_probs = model.predict_proba(X_test)[:, 1]

best_threshold = 0.0
best_f1 = 0.0

# We can search thresholds from 0.0 to 1.0 in small steps
for t in np.linspace(0, 1, 101):
    y_pred = (y_probs >= t).astype(int)
    score = f1_score(y_test, y_pred)
    if score > best_f1:
        best_f1 = score
        best_threshold = t

print(f"Best threshold: {best_threshold}")
print(f"Best F1 score: {best_f1}")

# When predicting on test data, use the best_threshold:
y_probs_test = model.predict_proba(X_test)[:, 1]
y_pred_test = (y_probs_test >= best_threshold).astype(int)


Best threshold: 0.54
Best F1 score: 0.1525829895894716


## Naive preprocessing (not my class)

In [17]:
data2= raw_data.drop(columns = ['session_id', 'DateTime', 'user_id','product_category_2'])
data2.dropna(inplace = True) # Just close your eyes and drop the rows with missing values

In [18]:
X_2 = data2.drop(columns = ['is_click'])
y_2 = data2['is_click']
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=42)

In [19]:
columns_to_d2 = ["product", "campaign_id", "webpage_id", "user_group_id", "gender", "product_category_1"]

X_train_2_d = pd.get_dummies(X_train_2, columns = columns_to_d2)
X_test_2_d = pd.get_dummies(X_test_2, columns = columns_to_d2)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(n_estimators=100, random_state=42,class_weight='balanced')
rf.fit(X_train_2_d, y_train_2)
y_pred_RF = rf.predict(X_test_2_d)



In [21]:
f1_score(y_test_2, y_pred_RF)

0.14298031865042174

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_2_d, y_train_2)

y_pred_LR = model.predict(X_test_2_d)
print(f1_score(y_test_2, y_pred_LR))

0.14430736693690518
